<a href="https://colab.research.google.com/github/awhitehouse1/DS4002-Project-2/blob/main/MergeData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# code below to merge the two separate data sets

In [6]:
import pandas as pd

# Load the datasets
covid_data = pd.read_excel("/content/COVID-19 Time Series Data.xlsx")
gdp_data = pd.read_excel("/content/gdp_per_capita.xlsx")

# Convert the 'Date' column in the COVID dataset to datetime format
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

# Filter the COVID data for dates between January 22, 2020, and January 22, 2022
start_date = '2020-01-22'
end_date = '2022-01-22'
covid_filtered = covid_data[(covid_data['Date'] >= start_date) & (covid_data['Date'] <= end_date)]

# Extract the year from the 'Date' column in the COVID dataset
covid_filtered['Year'] = covid_filtered['Date'].dt.year

# Select relevant columns from the COVID dataset
covid_filtered = covid_filtered[['Country/Region', 'Date', 'Year', 'Deaths']]

# Reshape the GDP data to have a 'Year' column, considering the year columns are integers
gdp_melted = gdp_data.melt(id_vars='CountryName', value_vars=[2020, 2021, 2022],
                           var_name='Year', value_name='GDP_per_Capita')

# Convert the 'Year' column to numeric
gdp_melted['Year'] = pd.to_numeric(gdp_melted['Year'])

# Merge the COVID dataset with the GDP dataset on 'Country/Region' and 'CountryName' and 'Year'
merged_data = pd.merge(covid_filtered, gdp_melted, how='inner', left_on=['Country/Region', 'Year'], right_on=['CountryName', 'Year'])

# Drop the 'CountryName' column (optional)
merged_data = merged_data.drop(columns=['CountryName'])

# Save the merged dataset as a CSV file for download
merged_data.to_csv('/content/merged_covid_gdp_data.csv', index=False)

# Provide the download link
from google.colab import files
files.download('/content/merged_covid_gdp_data.csv')




<ipython-input-6-b2eaa3f2f5ff>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_filtered['Year'] = covid_filtered['Date'].dt.year


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>